In [1]:
# IMC FIREBASE CALCULATOR

In [2]:
import firebase_admin
from firebase_admin import credentials

In [3]:
cred = credentials.Certificate("credentials.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://inf857---tp1---exo1---imc-default-rtdb.firebaseio.com/'
})